### Experimental notebook to validate if it even works in the first place

In [13]:
from sentence_transformers import SentenceTransformer
import torch
import dotenv
import os
from app.services.news_api import FinancialModelingPrepNews

torch.set_default_device("mps")
dotenv.load_dotenv()
api_key = os.getenv("FMP_API_KEY")


In [14]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="auto",
    api_key=os.environ["HF_TOKEN"],
)

result = client.feature_extraction(
    "Today is a sunny day and I will get some ice cream.",
    model="Qwen/Qwen3-Embedding-8B",
)
result

array([[ 0.01031494,  0.00448608,  0.00204468, ..., -0.00167847,
        -0.00396729, -0.00288391]], shape=(1, 4096), dtype=float32)

[
    {
        "symbol": null,
        "publishedDate": "2025-08-03 15:10:00",
        "publisher": "Market Watch",
        "title": "Top Fed officials unswayed by poor July jobs report, take wait-and-see approach to rate cuts",
        "image": "https://images.financialmodelingprep.com/news/top-fed-officials-unswayed-by-poor-july-jobs-report-20250803.jpg",
        "site": "marketwatch.com",
        "text": "Several senior Federal Reserve officials on Friday characterized the labor market as \u201csolid\u201d even after a poor July jobs report and appeared in no rush to lower U.S. interest rates.",
        "url": "https://www.marketwatch.com/story/top-fed-officials-unswayed-by-poor-july-jobs-report-take-wait-and-see-approach-to-rate-cuts-6b88d5df"
    },
    {
        "symbol": null,
        "publishedDate": "2025-08-03 14:20:06",
        "publisher": "Seeking Alpha",
        "title": "Wall Street Brunch: The BLS Also Does The CPI",
        "image": "https://images.financialmodelingpr